# Chapter 1
## Decision Trees 

In this notebook, we implement the decision tree in Python. We use the CIML Class Ratings dataset.


### Load Dataset

We use the CIML Class Ratings dataset, and preprocess it by:
* Converting integer ratings to binary like/hate (non-neg/neg) ratings
* Converting y/n attributes to boolean 0/1s

In [29]:
import pandas as pd
from collections import Counter
df = pd.read_csv("class_ratings.csv")
lh = df.copy()

# Convert integer ratings (labels) to non-neg/neg ratings (like-hate)
lh.loc[df["y"] >= 0, "y"] = 1
lh.loc[df["y"] < 0, "y"] = 0
# Convert feature values y/ns to 1/0s
lh.replace("y", 1, inplace=True)
lh.replace("n", 0, inplace=True)

# Create a attribute only dataframe
lh_attr = lh.drop(columns="y")

In [26]:
lh.head()

,y,easy,ai,sys,thy,morning
0,1,1,1,0,1,0
1,1,1,1,0,1,0
2,1,0,1,0,0,0
3,1,0,0,0,1,0
4,1,0,1,1,0,1


## Tree Data Structure

Our decision tree classifier only has binary splits. The left split corresponds with `False` value, right split corresponds to `True` value. To evaluate an instance, simply call `root.evaluate(data)`, which will recursive call its children.

There are two types of nodes: `Node` and `Leaf`. 

In [11]:
class Node:
    def __init__(self, attr, left, right):
        self.attr = attr
        self.left = left
        self.right = right
    def evaluate(self, data):
        if not data[self.attr]:
            return self.left.evaluate(data)
        else:
            return self.right.evaluate(data)

class Leaf:
    def __init__(self, value):
        self.attr = value
    
    def evaluate(self, data = None):
        return self.attr

## Training Algorithm

The `trainDecisionTree()` function will produce a decision tree using the `Node` and `Leaf` classes. This algorithm uses the greedy approach to select splits: at each iteration the cleanest attribute is selected. 

In [27]:
def trainDecisionTree(data, columns):
    label_counts = Counter(data["y"])
    freq, num_freq = maxDict(label_counts) # the most frequent label in data
    if num_freq == len(data["y"]): # if data contains only one label
        return Leaf(freq)
    if len(columns) == 0: # if tree ran out of data columns
        return Leaf(freq)
    scores = {}
    for column in columns:
        c0 = data[data[column] == 0]
        c1 = data[data[column] == 1]
        count_c0 = Counter(c0["y"])
        count_c1 = Counter(c1["y"])
        freq_c0, num_freq_c0 = maxDict(count_c0)
        freq_c1, num_freq_c1 = maxDict(count_c1)
        scores[column] = (num_freq_c0 + num_freq_c1) / len(data)
    best_column, _ = maxDict(scores)
    columns.remove(best_column)
    left = trainDecisionTree(data[data[best_column] == 0], columns)
    right = trainDecisionTree(data[data[best_column] == 1], columns)
    return Node(best_column, left, right)


In [12]:
# Returns the key of the maximum value in dict. 
# If dict is empty, return (None, 0)
def maxDict(count):
    if len(count) == 0:
        return (None, 0)
    m = max(count, key=count.get)
    return (m, count[m])

## Evaluation

In [14]:
def print_tree(node, _  = "", _last=True):
    print(_ , "`- " if _last else "|- ", node.attr, sep="")
    _  += "   " if _last else "|  "
    
    if isinstance(node, Node):
        # left side
        print_tree(node.left, _ , False)
        # right side
        print_tree(node.right, _ , True)


In [16]:
root = trainDecisionTree(lh, list(lh.columns)[1:])

In [18]:
print_tree(root)

`- sys
   |- 1
   `- easy
      |- ai
      |  |- 0
      |  `- thy
      |     |- morning
      |     |  |- None
      |     |  `- 1
      |     `- None
      `- 0


In [28]:
num_correct = 0
test_size = len(lh_attr)
for i in range(test_size):
    features = dict(lh_attr.loc[i])
    label = lh.loc[i]["y"]
    predicted = root.evaluate(features)
    if predicted == label:
        num_correct += 1
    print("{0}\tLabel: {1}, Predicted: {2}".format(features, label, predicted))
print("Acc: {0}/{1} = {2}".format(num_correct, test_size, num_correct/test_size))



{'easy': 1, 'ai': 1, 'sys': 0, 'thy': 1, 'morning': 0}	Label: 1, Predicted: 1
{'easy': 1, 'ai': 1, 'sys': 0, 'thy': 1, 'morning': 0}	Label: 1, Predicted: 1
{'easy': 0, 'ai': 1, 'sys': 0, 'thy': 0, 'morning': 0}	Label: 1, Predicted: 1
{'easy': 0, 'ai': 0, 'sys': 0, 'thy': 1, 'morning': 0}	Label: 1, Predicted: 1
{'easy': 0, 'ai': 1, 'sys': 1, 'thy': 0, 'morning': 1}	Label: 1, Predicted: 1
{'easy': 1, 'ai': 1, 'sys': 0, 'thy': 0, 'morning': 0}	Label: 1, Predicted: 1
{'easy': 1, 'ai': 1, 'sys': 0, 'thy': 1, 'morning': 0}	Label: 1, Predicted: 1
{'easy': 0, 'ai': 1, 'sys': 0, 'thy': 1, 'morning': 0}	Label: 1, Predicted: 1
{'easy': 0, 'ai': 0, 'sys': 0, 'thy': 0, 'morning': 1}	Label: 1, Predicted: 1
{'easy': 1, 'ai': 0, 'sys': 0, 'thy': 1, 'morning': 1}	Label: 1, Predicted: 1
{'easy': 0, 'ai': 1, 'sys': 0, 'thy': 1, 'morning': 0}	Label: 1, Predicted: 1
{'easy': 1, 'ai': 1, 'sys': 1, 'thy': 1, 'morning': 1}	Label: 1, Predicted: 0
{'easy': 1, 'ai': 1, 'sys': 1, 'thy': 0, 'morning': 1}	Label: 0,